Лабораторная работа №4
"Построение рекомендательной системы на основе коллаборативной фильтрации"

Цель:

Познакомиться с базовыми подходами к построению рекомендательных систем. Научиться использовать пользовательские взаимодействия для генерации рекомендаций.

📋 Часть 1: Теоретическое введение

#Рекомендательная система
Рекомендательная система — это алгоритм или система, которая предлагает пользователям товары, услуги или контент (например, фильмы, музыку, статьи) на основе их предпочтений, поведения или данных других пользователей. Такие системы используются в интернет-магазинах (Amazon), стриминговых сервисах (Netflix, Spotify) и социальных сетях.

#Content-based filtering (контентный подход):
Рекомендации основаны на характеристиках самих объектов (например, жанр фильма, описание товара) и предпочтениях пользователя. Если пользователь смотрел комедии, система предложит другие комедии.
Преимущества: Не зависит от данных других пользователей, работает с новыми элементами.
Недостатки: Требует качественных данных о характеристиках объектов, может ограничивать разнообразие рекомендаций.

#Collaborative filtering (коллаборативная фильтрация):
Рекомендации основаны на взаимодействиях пользователей с объектами. Если два пользователя имеют схожие оценки или покупки, система предложит им похожие товары.
Преимущества: Не требует данных о характеристиках объектов, может выявлять неожиданные связи.
Недостатки: Проблема "холодного старта" (не работает для новых пользователей/объектов), требует больших данных.

#Матрица взаимодействий
Матрица взаимодействий — это таблица, где строки представляют пользователей, столбцы — объекты (например, товары или фильмы), а значения — взаимодействия (оценки, покупки, просмотры). Обычно матрица разреженная (sparse), так как пользователи взаимодействуют только с небольшим количеством объектов. Пример:
Пользователь 1 оценил фильм A на 5, фильм B на 3, а фильм C не смотрел → строка: [5, 3, NaN].
Используется в коллаборативной фильтрации для анализа схожести пользователей или объектов.

#User2User (пользователь-пользователь):
Подход в коллаборативной фильтрации, где система находит пользователей, похожих на целевого (по их взаимодействиям), и рекомендует объекты, которые понравились этим похожим пользователям.
Пример: Если пользователь A и B оба высоко оценили фильмы X и Y, а B также оценил фильм Z, то A порекомендуют Z.
Проблемы: Высокая вычислительная сложность при большом числе пользователей.

#Item2Item (объект-объект):
Подход в коллаборативной фильтрации, где система находит объекты, похожие на те, с которыми пользователь уже взаимодействовал, и рекомендует их.
Пример: Если пользователь A посмотрел фильм X, а фильм X часто смотрят вместе с фильмом Y, то порекомендуют Y.
Преимущества: Более масштабируемый, так как объектов обычно меньше, чем пользователей, и схожесть между объектами меняется реже.

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from collections import defaultdict


df = pd.read_csv('DS_uni_labs/lab4/ratings.csv')
# Нормализация рейтингов
user_means = df.groupby('userId')['rating'].mean()
df['normalized_rating'] = df.apply(lambda x: x['rating'] - user_means[x['userId']], axis=1)

# Задание 1
user_item_matrix = pd.pivot_table(df, values='normalized_rating', index='userId', columns='movieId', fill_value=0)
user_item_matrix_raw = pd.pivot_table(df, values='rating', index='userId', columns='movieId', fill_value=0)
user_item_matrix_np = user_item_matrix.to_numpy()

# Задание 2
user_similarity = cosine_similarity(user_item_matrix_np)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

target_user_id = 1
target_user_idx = user_item_matrix.index.get_loc(target_user_id)
similar_users = user_similarity_df.iloc[target_user_idx].sort_values(ascending=False)[1:6]
similar_user_ids = similar_users.index

target_user_ratings = user_item_matrix_raw.loc[target_user_id]
unrated_movies = target_user_ratings[target_user_ratings == 0].index
recommended_movies = user_item_matrix_raw.loc[similar_user_ids, unrated_movies]
recommended_movie = recommended_movies.mean().sort_values(ascending=False).index[0]

print(f"Рекомендованный фильм для userId={target_user_id} (User-based с нормализацией): movieId={recommended_movie}")

# Задание 3
item_user_matrix_np = user_item_matrix_np.T
item_similarity = cosine_similarity(item_user_matrix_np)
item_similarity_df = pd.DataFrame(item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)

rated_movie_id = 16
rated_movie_idx = user_item_matrix.columns.get_loc(rated_movie_id)
similar_movies = item_similarity_df.iloc[rated_movie_idx].sort_values(ascending=False)[1:6]
similar_movie_ids = similar_movies.index

print(f"Похожие фильмы на movieId={rated_movie_id} (Item-based с нормализацией): {similar_movie_ids.tolist()}")

# свдшка
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

svd = SVD(n_factors=50, n_epochs=20, lr_all=0.005, reg_all=0.02, random_state=42)
svd.fit(trainset)

# Задание 4
def get_top_n_svd(user_id, n=5):
    all_movies = user_item_matrix.columns
    rated_movies = user_item_matrix_raw.loc[user_id][user_item_matrix_raw.loc[user_id] > 0].index
    unrated_movies = [movie for movie in all_movies if movie not in rated_movies]
    
    predictions = [(movie, svd.predict(user_id, movie).est) for movie in unrated_movies]
    predictions.sort(key=lambda x: x[1], reverse=True)
    top_n = predictions[:n]
    return [movie for movie, _ in top_n]

top_5_svd = get_top_n_svd(target_user_id, n=5)
print(f"Топ-5 рекомендованных фильмов для userId={target_user_id} (SVD): {top_5_svd}")

Рекомендованный фильм для userId=1 (User-based с нормализацией): movieId=2329
Похожие фильмы на movieId=16 (Item-based с нормализацией): [7158, 5131, 5534, 34767, 48711]
Топ-5 рекомендованных фильмов для userId=1 (SVD): [2329, 88125, 800, 923, 4226]
